# CS445: Term Project

#### Victor Weeks & Josh Lutz

Note: I think I am going to explore changing my idea to taking current inventory info and deciding how best to make a profit

#### Get material data

In [47]:
import urllib.request, urllib.error, urllib.parse
api_key = "Bearer 51DD34C0-ABC4-E54C-A3F2-8E7500AE5F74705C18C6-B5BD-431A-8A6A-304EB4F2A24A"
headers = {'Authorization': api_key}

In [48]:
def getAccountMaterials():
        """Makes a request on the GW2 API."""
        url = 'https://api.guildwars2.com/v2/account/materials/'
        r = urllib.request.Request(url, headers=headers)
        if 'Cookie' not in headers:
            resp = urllib.request.urlopen(r)
            if 'set-cookie' in resp.headers:
                headers['Cookie'] = resp.headers['set-cookie'].split(';', 1)[0]
            return json.loads(resp.read())
        return json.loads(urllib.request.urlopen(r).read())

In [62]:
materials = getAccountMaterials()

In [70]:
materials[0]

{'category': 5, 'count': 56, 'id': 12134}

In [73]:
def saveMaterials(materials):
    playerMats = {}
    for mat in materials:        
        if mat['id'] in itemIDs:
            playerMats[mat['id']] = mat['count']
    with open('playerMaterials.txt', 'w') as outFile:
        json.dump(playerMats, outFile)

In [74]:
saveMaterials(materials)

In [76]:
matData = open('playerMaterials.txt')
matData = json.load(matData)
matData['19725']

151

## Introduction

In [38]:
from gw2spidy import Gw2Spidy as spidy # allows retrieval of market information
import numpy as np
import json
import random
import neuralnetworksA4 as nn

### Data

In [2]:
# Tier 1
juteScrap = spidy.getItemData(19718)
boltOfJute = spidy.getItemData(19720)
copperOre = spidy.getItemData(19697)
copperIngot = spidy.getItemData(19680)
rawhideLeatherSection = spidy.getItemData(19719)
stretchedRawhideLeatherSquare = spidy.getItemData(19738)
greenWoodLog = spidy.getItemData(19723)
greenWoodPlank = spidy.getItemData(19710)

In [3]:
# Tier 2
woolScrap = spidy.getItemData(19739)
boltOfWool = spidy.getItemData(19740)
ironOre = spidy.getItemData(19699)
ironIngot = spidy.getItemData(19683)
silverOre = spidy.getItemData(19703)
silverIngot = spidy.getItemData(19687)
thinLeatherSection = spidy.getItemData(19728)
curedThinLeatherSquare = spidy.getItemData(19733)
softWoodLog = spidy.getItemData(19726)
softWoodPlank = spidy.getItemData(19713)

In [4]:
# Tier 3
cottonScrap = spidy.getItemData(19741)
boltOfCotton = spidy.getItemData(19742)
goldOre = spidy.getItemData(19698)
goldIngot = spidy.getItemData(19682)
coarseLeatherSection = spidy.getItemData(19730)
curedCoarseLeatherSquare = spidy.getItemData(19734)
seasonedWoodLog = spidy.getItemData(19727)
seasonedWoodPlank = spidy.getItemData(19714)

In [5]:
# Tier 4
linenScrap = spidy.getItemData(19743)
boltOfLinen = spidy.getItemData(19744)
platinumOre = spidy.getItemData(19702)
platinumIngot = spidy.getItemData(19686)
ruggedLeatherSection = spidy.getItemData(19731)
curedRuggedLeatherSquare = spidy.getItemData(19736)
hardWoodLog = spidy.getItemData(19724)
hardWoodPlank = spidy.getItemData(19711)

In [6]:
# Tier 5
silkScrap = spidy.getItemData(19748)
boltOfSilk = spidy.getItemData(19747)
mithrilOre = spidy.getItemData(19700)
mithrilIngot = spidy.getItemData(19684)
thickLeatherSection = spidy.getItemData(19729)
curedThickLeatherSection = spidy.getItemData(19735)
elderWoodLog = spidy.getItemData(19722)
elderWoodPlank = spidy.getItemData(19709)

In [7]:
# Tier 6
gossamerScrap = spidy.getItemData(19745)
boltOfGossamer = spidy.getItemData(19746)
orichalcumOre = spidy.getItemData(19701)
orichalcumIngot = spidy.getItemData(19685)
hardenedLeatherSection = spidy.getItemData(19732)
curedHardenedLeatherSquare = spidy.getItemData(19737)
ancientWoodLog = spidy.getItemData(19725)
ancientWoodPlank = spidy.getItemData(19712)

In [3]:
itemIDs = (19718,19720,19697,19680,19719,19738,19723,19710,
           19739,19740,19699,19683,19703,19687,19728,19733,19726,19713,
           19741,19742,19698,19682,19730,19734,19727,19714,
           19743,19744,19702,19686,19731,19736,19736,19724,19711,
           19748,19747,19700,19684,19729,19735,19722,19709,
           19745,19746,19701,19685,19732,19737,19725,19712)

In [7]:
def saveItemData(itemIDs):
    allItems = {}
    for id in itemIDs:
        itemData = {}
        item = spidy.getItemData(id)
        itemID = item['data_id']
        itemData['name'] = item['name']
        itemData['price_last_changed'] = item['price_last_changed']
        itemData['max_offer_unit_price'] = item['max_offer_unit_price']
        itemData['min_sale_unit_price'] = item['min_sale_unit_price']
        itemData['offer_availability'] = item['offer_availability']
        itemData['sale_availability'] = item['sale_availability']
        itemData['sale_price_change_last_hour'] = item['sale_price_change_last_hour']
        itemData['offer_price_change_last_hour'] = item['offer_price_change_last_hour']
        allItems[itemID] = itemData
    return allItems
        

In [11]:
# Only do this once, don't overwrite old data
if False:
    with open('itemData.txt', 'w') as outFile:
        json.dump(saveItemData(itemIDs), outFile)

In [12]:
itemData = open('itemData.txt')
lastItemData = json.load(itemData)
lastItemData['19725']

{'max_offer_unit_price': 152,
 'min_sale_unit_price': 171,
 'name': 'Ancient Wood Log',
 'offer_availability': 84151,
 'offer_price_change_last_hour': -5,
 'price_last_changed': '2018-05-04 23:20:31 UTC',
 'sale_availability': 491002,
 'sale_price_change_last_hour': 0}

In [48]:
def savePortfolio(itemIDs):
    allItems = {}
    for id in itemIDs:
        itemPortfolio = {}
        itemPortfolio['num_units_possessed'] = 0
        itemPortfolio['max_unit_buy_price'] = 0
        allItems[id] = itemPortfolio
    return allItems

In [14]:
# Only do this once
if False:
    with open('portfolio.txt', 'w') as portfolio:
        json.dump(savePortfolio(itemIDs), portfolio)

In [15]:
portfolio = open('portfolio.txt')
currPortfolio = json.load(portfolio)
currPortfolio['19725']

{'max_unit_buy_price': 0, 'num_units_possessed': 0}

## Methods

In [41]:
# This method acts as the reinforcement and accounts for the 15% handling cost for selling items on the market
def checkForProfit(price_sold, price_bought):
    profit = (price_sold - (price_sold * 0.15)) - price_bought
    if profit > 0:
        return 1
    elif profit < 0:
        return -1
    else:
        return 0

Note: General training of net to be exposed to various states

The state for this training is
(max_offer_change, min_sale_change, offer_availability_change, sale_availability_change, units_possessed, unit_buy_price).
The first four compare state at timepoint 2 to state at timepoint 1. A value of 1 indicates t2 > t1, 0 indicates no change, and -1 indicates t2 < t1.
units_possessed is a value between 0 and 10 and unit_price is int between 1 and 100

In [37]:
from copy import copy

actions = (-1, 0, 1) # Possible actions (sale, none, buy)

# time 1
offer_price_T1 = random.randint(1, 100)
sale_price_T1 = random.randint(1, 100)
offer_avail_T1 = random.randint(1, 100)
sale_avail_T1 = random.randint(1, 100)

# time 2
offer_price_T2 = random.randint(1, 100)
sale_price_T2 = random.randint(1, 100)
offer_avail_T2 = random.randint(1, 100)
sale_avail_T2 = random.randint(1, 100)


def getTimeDifference(value_T1, value_T2):
    if value_T2 > value_T1:
        return 1
    elif value_T2 < value_T1:
        return -1
    else:
        return 0

def reinforcement(price_sold, price_bought):
    return checkForProfit(price_sold, price_bought)

def initialState():
    # (max_offer_change, min_sale_change, offer_availability_change, sale_availability_change)
    return np.array([getTimeDifference(offer_price_T1, offer_price_T2), getTimeDifference(sale_price_T1, sale_price_T2),
                     getTimeDifference(offer_avail_T1, offer_avail_T2), getTimeDifference(sale_avail_T1, sale_avail_T2),
                     random.randint(0,10), random.randint(1,100)])

def nextState(s, a):
    s = copy(s)
    s[4] += a
    if a == 1:
        s[5] = max(s[5], offer_price_T2)
    return s
        
def epsilonGreedy(nnetQ, state, actions, epsilon):
    if np.random.uniform() < epsilon:
        # Random Move
        action = np.random.choice(actions)
    else:
        # Greedy Move
        Qs = [nnetQ.use(np.hstack((state,a)).reshape((1, -1))) for a in actions]
        ai = np.argmax(Qs)
        action = actions[ai]
    Q = nnetQ.use(np.hstack((state,action)).reshape((1, -1)))
    return action, Q

def makeSamples(nnet, initialStateF, nextStateF, reinforcementF,
                validActions, numSamples, epsilon):

    X = np.zeros((numSamples, nnet.ni))
    R = np.zeros((numSamples, 1))
    Qn = np.zeros((numSamples, 1))

    s = initialStateF()
    s = nextStateF(s, 0)        # Update state, sn from s and a
    a, _ = epsilonGreedy(nnet, s, validActions, epsilon)

    # Collect data from numSamples steps
    for step in range(numSamples):
        sn = nextStateF(s, a)        # Update state, sn from s and a
        rn = reinforcementF(s, sn)   # Calculate resulting reinforcement
        an, qn = epsilonGreedy(nnet, sn, validActions, epsilon) # Forward pass for time t+1
        X[step, :] = np.hstack((s,a))
        R[step, 0] = rn
        Qn[step, 0] = qn
        # Advance one time step
        s, a = sn, an

    return (X, R, Qn)



## Results

## Conclusions

## References